<a href="https://colab.research.google.com/github/Loga19818eeanvesh/CNN_Models/blob/main/2019_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.2.0

In [ ]:
import tensorflow as tf 
print(tf.__version__)

2.2.0


In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import DepthwiseConv2D,Conv2D,BatchNormalization,GlobalAveragePooling2D,Flatten,Add,InputLayer,ReLU
from tensorflow.keras.activations import relu,softmax
from tensorflow.nn import relu6,conv2d,depthwise_conv2d

In [34]:
class DottleneckDepthSeparableConvolution(Model):
  def __init__(self,t,c,f):
    super().__init__()
    self.expansion = Conv2D(filters=(t*c),kernel_size=1)
    self.depthwise_conv = DepthwiseConv2D(kernel_size=3,padding='same')
    self.projection = Conv2D(filters=f,kernel_size=1)
    self.relu_activation = ReLU()
    self.add = Add()

  def call(self,inputs):
    x = self.expansion(inputs)
    x = self.relu_activation(x)
    x = self.depthwise_conv(x)
    x = self.relu_activation(x)
    x = self.projection(x)
    output = x
    if(x.shape==inputs.shape):
      output = self.add([x,inputs])
      
    return output

In [33]:
class DottleneckDepthSeparableConvolutionStride2Block(Model):
  def __init__(self,t,c,f):
    super().__init__()
    self.channels = t*c
    self.expansion = Conv2D(filters=(t*c),kernel_size=1)
    self.depthwise_conv = depthwise_conv2d
    self.projection = Conv2D(filters=f,kernel_size=1)
    self.relu_activation = ReLU()
    f_init = tf.random_normal_initializer()
    self.filter_init = tf.Variable(initial_value=f_init(shape=(3,3,(t*c),1) , dtype="float32") , trainable=True)
    self.bias = tf.nn.bias_add
    self.b = tf.Variable(tf.zeros(shape = [(t*c),]), dtype="float32")

  def call(self,inputs):
    x = self.expansion(inputs)
    x = self.relu_activation(x)
    x = self.depthwise_conv(x,filter=self.filter_init,strides = [1, 2, 2, 1],padding=[[0, 0], [1,0],[1,0], [0, 0]])
    x = self.bias(x, self.b)
    x = self.relu_activation(x)
    x = self.projection(x)

    return x

In [35]:
class MobileNetBlock(Model):
  def __init__(self,t,c,f,n):
    super().__init__()
    self.stride2 = DottleneckDepthSeparableConvolutionStride2Block(t=t,c=c,f=f)
    self.nl = n-1
    self.layers1 = {}
    for i in range(n-1):
      self.layers1["stride1"+str(i)] = DottleneckDepthSeparableConvolution(t=t,c=c,f=f)

  def call(self,inputs):
    x = self.stride2(inputs)
    for i in range(self.nl):
      x = self.layers1["stride1"+str(i)](x)
    return x

In [36]:
class MobileNetV2(Model):
  def __init__(self,input_shape,noof_classes):
    super().__init__()
    self.input1 = InputLayer(input_shape=input_shape)
    f_init = tf.random_normal_initializer()
    self.filter_init = tf.Variable(initial_value=f_init(shape=(3,3,3,32) , dtype="float32") , trainable=True)
    self.bias = tf.nn.bias_add
    self.relu_activation = ReLU()
    self.b = tf.Variable(tf.zeros(shape = [32,]), dtype="float32")
    self.conv1 = conv2d
    self.bottleneck1 = DottleneckDepthSeparableConvolution(t=1,c=32,f=16)
    self.mobile_net_block1 = MobileNetBlock(t=6,c=16,f=24,n=2)
    self.mobile_net_block2 = MobileNetBlock(t=6,c=24,f=32,n=3)
    self.mobile_net_block3 = MobileNetBlock(t=6,c=32,f=64,n=4)
    self.bottleneck2 = DottleneckDepthSeparableConvolution(t=6,c=64,f=96)
    self.bottleneck3 = DottleneckDepthSeparableConvolution(t=6,c=64,f=96)
    self.bottleneck4 = DottleneckDepthSeparableConvolution(t=6,c=64,f=96)
    self.mobile_net_block5 = MobileNetBlock(t=6,c=96,f=160,n=3)
    self.bottleneck5 = DottleneckDepthSeparableConvolution(t=6,c=160,f=320)
    self.conv2 = Conv2D(kernel_size=1,filters=1280)
    self.global_avg_pool = GlobalAveragePooling2D()
    self.conv3 = Conv2D(kernel_size=1,filters=noof_classes)
    self.flattten = Flatten()
    self.classifier = softmax

  def call(self,inputs):
    x = self.input1(inputs)
    x = self.conv1(inputs,filters=self.filter_init,strides = [1, 2, 2, 1],padding= [[0, 0], [1,0],[1,0], [0, 0]])
    x = x = self.bias(x, self.b)
    x = self.relu_activation(x)
    x = self.bottleneck1(x)
    x = self.mobile_net_block1(x)
    x = self.mobile_net_block2(x)
    x = self.mobile_net_block3(x)
    x = self.bottleneck2(x)
    x = self.bottleneck3(x)
    x = self.bottleneck4(x)
    x = self.mobile_net_block5(x)
    x = self.bottleneck5(x)
    x = self.conv2(x)
    x = self.global_avg_pool(x)
    x = self.conv3(x)
    x = self.flattten(x)
    x = self.classifier(x)

    return x
    

In [39]:
MobileNetV2_model = MobileNetV2(input_shape=(None,224,224,3),noof_classes=1000)

In [ ]:
MobileNetV2_model.build(input_shape=(None,224,224,3)) 

In [ ]:
MobileNetV2_model.summary()